In [2]:
import pandas as pd
import matplotlib.pyplot as plt

#data = pd.read_stata("./uniswap0826.dta")

In [3]:
#file_name = "240103_lp_data_with_sc_and_rec_usage.csv"
#file_name = "240117_lp_data_with_sc_and_rec_usage.csv"
file_name = "240124_lp_data_with_sc_and_rec_usage.csv"
data = pd.read_csv("../01_create/" + file_name)
data = data.sort_values(by=['pool_addr', 'liquidity_provider', 'week'])
price_info = pd.read_csv("../01_create/" + "240103_weekly_pool_agg_results.csv")
def qtile_25(x):
    return x.quantile(0.25)
def qtile_75(x):
    return x.quantile(0.75)
def qtile_10(x):
    return x.quantile(0.10)
def qtile_90(x):
    return x.quantile(0.90)

In [4]:
data["liquidity_provider"].nunique()

38105

In [5]:
data["cum_in"] = data.groupby(['pool_addr', 'liquidity_provider'])["amount_input"].cumsum()

In [6]:
#data = data_bak.copy()
data_bak = data.copy()

In [7]:
# Filter out the large enough ones 
#data_bak = data.copy()
ok_lp_ids = data[data["cum_in"] > 10]["liquidity_provider"].unique()
data = data[data["liquidity_provider"].isin(ok_lp_ids)].copy()

In [8]:
data["liquidity_provider"].nunique()

25142

In [9]:
ok_lp_ids_cond2 = data[data["position_cnt"] < 8]["liquidity_provider"].unique()

In [10]:
# Third filtering
data["week"] = pd.to_datetime(data['week'])
data["last_week"] = data.groupby(['pool_addr', 'liquidity_provider'])["week"].shift(1)
data["last_two_week"] = data.groupby(['pool_addr', 'liquidity_provider'])["week"].shift(2)
data["last_three_week"] = data.groupby(['pool_addr', 'liquidity_provider'])["week"].shift(3)
data["last_four_week"] = data.groupby(['pool_addr', 'liquidity_provider'])["week"].shift(4)

In [11]:
data['continue_flag_con1'] = data["week"] - data["last_week"] == '7 days'
data['continue_flag_con2'] = data["last_week"] - data["last_two_week"] == '7 days'
data['continue_flag_con3'] = data["last_two_week"] - data["last_three_week"] == '7 days'
data['continue_flag_con4'] = data["last_three_week"] - data["last_four_week"] == '7 days'

In [12]:
data_cond1 = data[data["continue_flag_con1"]].copy()
data_cond2 = data_cond1[data_cond1["continue_flag_con2"]].copy()
data_cond3 = data_cond2[data_cond2["continue_flag_con3"]].copy()
data_cond4 = data_cond3[data_cond3["continue_flag_con4"]].copy()

In [27]:
data = data_cond4.copy()

In [28]:
data = data[data["liquidity_provider"].isin(ok_lp_ids_cond2)].copy()

In [29]:
data["liquidity_provider"].nunique()

19959

In [30]:
#data = data[data["sc_usage_avg"] > 0]

In [31]:
data.shape

(541453, 27)

In [33]:
data = data_bak.copy()

In [22]:
pool_addrs = ['0x11b815efb8f581194ae79006d24e0d814b7697f6',
                  '0x4e68ccd3e89f51c3074ca5072bbac773960dfa36',
                  '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
                  '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8']
for pool_addr in pool_addrs:
    price_info_given_pool = price_info[price_info["pool_address"] == pool_addr].copy()
    data_given_pool = data[data["pool_addr"]==pool_addr].copy()
    data_given_pool["cum_overall_roi"] = data_given_pool.groupby(['liquidity_provider'])["overall_roi"].cumprod()
    # get the LP-SC mapping:
    lp_sc_mapping = pd.read_csv("../01_create/" + pool_addr + "lp_using_verified_sc.csv")[["liquidity_provider","week"]]
    data_given_pool["week"] = pd.to_datetime(data['week']).astype(str)
    lp_sc_mapping["sc_type"] = "verified"
    with_sc_type = data_given_pool.merge(lp_sc_mapping, how='left', on=['liquidity_provider', 'week'])
    regular_sc_usage = (with_sc_type["sc_usage_avg"] > 0) & (with_sc_type["sc_type"].isna())
    with_sc_type.loc[regular_sc_usage, "sc_type"] = 'non_verified'
    with_sc_type["sc_type"] = with_sc_type["sc_type"].fillna("not_sc")

    # grouped_temp_data = with_sc_type.groupby(["sc_type", "week"]).agg(
    #     cum_overall_mean=('cum_overall_roi', 'mean'),
    #     cum_overall_median=('cum_overall_roi', 'median'),
    #     cum_overall_q10=('cum_overall_roi', qtile_10),
    #     cum_overall_q25=('cum_overall_roi', qtile_25),
    #     cum_overall_q90=('cum_overall_roi', qtile_90),
    #     cum_overall_q75=('cum_overall_roi', qtile_75),
    # ).reset_index()
    grouped_temp_data = with_sc_type.groupby(["sc_type", "week"]).agg(
            daily_overall_avg=('overall_roi', 'mean'),
            daily_amt_avg=('amt_roi', 'mean'),
            daily_fee_avg=('fee_roi', 'mean'),
            daily_overall_median=('overall_roi', 'median'),
            daily_amt_median=('amt_roi', 'median'),
            daily_fee_median=('fee_roi', 'median'),
            daily_overall_q75=('overall_roi', qtile_75),
            daily_amt_q75=('amt_roi', qtile_75),
            daily_fee_q75=('fee_roi', qtile_75),
            daily_overall_q25=('overall_roi', qtile_25),
            daily_amt_q25=('amt_roi', qtile_25),
            daily_fee_q25=('fee_roi', qtile_25),
            daily_overall_q10=('overall_roi', qtile_10),
            daily_amt_q10=('amt_roi', qtile_10),
            daily_fee_q10=('fee_roi', qtile_10),
            daily_overall_q90=('overall_roi', qtile_90),
            daily_amt_q90=('amt_roi', qtile_90),
            daily_fee_q90=('fee_roi', qtile_90),
    ).reset_index()
    
    grouped_temp_data['date'] = pd.to_datetime(grouped_temp_data['week'])
    grouped_temp_data.set_index(['date'], inplace=True)
    #col_names = ['cum_overall_mean', 'cum_overall_median', 'cum_overall_q10', 'cum_overall_q25', 'cum_overall_q75', 'cum_overall_q90']
    col_names = ['daily_overall_avg', 'daily_amt_avg', 'daily_fee_avg', 'daily_overall_median', 'daily_amt_median', 'daily_fee_median',
                 'daily_overall_q75', 'daily_amt_q75', 'daily_fee_q75', 'daily_overall_q25', 'daily_amt_q25', 'daily_fee_q25',
                 'daily_overall_q10', 'daily_amt_q10', 'daily_fee_q10', 'daily_overall_q90', 'daily_amt_q90', 'daily_fee_q90'
                 ]
    for temp_col_name in col_names:
        import matplotlib.pyplot as plt
        # true_human_cond = grouped_temp_data['lp_type'] == 'rec_only'
        # false_human_cond = grouped_temp_data['lp_type'] == 'manual_only'
        # mixed_human_cond = grouped_temp_data['lp_type'] == 'rec_and_manual_mixed'
        # sc_cond = grouped_temp_data['lp_type'] == 'sc_only'
        # data_true = grouped_temp_data[true_human_cond].copy().reset_index()
        # data_false = grouped_temp_data[false_human_cond].copy().reset_index()
        # data_mixed = grouped_temp_data[mixed_human_cond].copy().reset_index()
        verified_sc_cond = grouped_temp_data['sc_type'] == 'verified'
        non_verified_cond = grouped_temp_data['sc_type'] == 'non_verified'
        human_cond = grouped_temp_data['sc_type'] == 'not_sc'
        #sc_cond = grouped_temp_data['lp_type'] == 'sc_only'
        data_true = grouped_temp_data[verified_sc_cond].copy().reset_index()
        data_false = grouped_temp_data[non_verified_cond].copy().reset_index()
        data_human = grouped_temp_data[human_cond].copy().reset_index()
        #data_sc = grouped_temp_data[sc_cond].copy().reset_index()
        # 'alive_positions', 'money_in_pool_total', 'money_in_pool_avg',
        #        'money_in_pool_median', 'daily_price', 'overall_earning_count',
        #        'overall_in_range', 'age_avg', 'age_median', 'in_range_perc',
        #        'overall_earning_perc'],
        # Creating a twin Y-axis for the 'daily_price' column
        fig, ax1 = plt.subplots(figsize=(24, 8))
        # Plotting 'daily_fee_avg_winsorized' on the primary Y-axis
        ax1.plot(data_true['date'], data_true[temp_col_name], label='Verified SC', color='blue', linestyle='--',)
        ax1.plot(data_human['date'], data_human[temp_col_name], label='Human Pos', color='green', linestyle='--')
        ax1.plot(data_false['date'], data_false[temp_col_name], label='Non-Verified SC', color='orange', linestyle='--')
        #ax1.plot(data_mixed['date'], data_mixed[temp_col_name], label='Manual and Rec Mixed Strategy', color='red', linestyle='--')

        #ax1.plot(data_non_sc['week'], data_non_sc[col_name], label='Human', color='blue', linestyle='--',)
        #ax1.plot(data_sc['week'], data_sc[col_name], label='SC', color='green', linestyle='--')
        # ax1.plot(data_true['date'], data_true[col_name], label='Rec ', color='green', linestyle='--')
        # ax1.plot(data_nonrec_and_rec['date'], data_nonrec_and_rec[temp_col_name+"_x"]/data_nonrec_and_rec[temp_col_name+"_y"], label='Non-rec vs Rec only', color='orange', linestyle='--')
        # ax1.plot(data_mixed_and_rec['date'], data_mixed_and_rec[temp_col_name+"_x"]/data_mixed_and_rec[temp_col_name+"_y"], label='Mixed vs Rec only', color='red', linestyle='--')
        #ax1.axhline(y=1.0, color='blue', label='Baseline = 1.0 (Earns Money)')

        #ax1.plot(data_false['date'], data_false[col_name], label='Manual', color='orange', linestyle='--')
        #ax1.plot(data_sc['date'], data_sc[col_name], label='SC', color='green', linestyle='--')

        #ax1.plot(data_false['date'], data_false[col_name], label='Recommended = False', color='red', linestyle='--', alpha=0.7)
        ax1.set_xlabel('Date')
        ax1.set_ylabel(temp_col_name, color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.legend(loc='upper left')
        ax1.grid(True)

        plt.title(temp_col_name + ' in ' + pool_addr + ' Over Time')
        plt.savefig("./graph_roi_0123/" + temp_col_name + '_' + pool_addr + ".png")
        plt.close()
    # Show the plot


In [34]:
pool_addrs = ['0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
                  '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8']
for pool_addr in pool_addrs:
    price_info_given_pool = price_info[price_info["pool_address"] == pool_addr].copy()
    data_given_pool = data[data["pool_addr"]==pool_addr].copy()
    data_given_pool["cum_overall_roi"] = data_given_pool.groupby(['liquidity_provider'])["overall_roi"].cumprod()
    # get the LP-SC mapping:
    lp_sc_mapping = pd.read_csv("../01_create/" + pool_addr + "lp_using_verified_sc.csv")[["liquidity_provider","week"]]
    data_given_pool["week"] = pd.to_datetime(data['week']).astype(str)
    lp_sc_mapping["sc_type"] = "verified"
    with_sc_type = data_given_pool.merge(lp_sc_mapping, how='left', on=['liquidity_provider', 'week'])
    regular_sc_usage = (with_sc_type["sc_usage_avg"] > 0) & (with_sc_type["sc_type"].isna())
    with_sc_type.loc[regular_sc_usage, "sc_type"] = 'non_verified'
    with_sc_type["sc_type"] = with_sc_type["sc_type"].fillna("not_sc")
    larger_than_zero = with_sc_type[with_sc_type["amount_input"] > 0].copy()
    print(larger_than_zero.groupby(["sc_type"])["amount_input"].describe())

                count          mean           std           min           25%  \
sc_type                                                                         
non_verified    313.0  4.643720e+06  1.125618e+07  2.510016e-02  40781.069153   
not_sc        14677.0  8.196622e+05  6.654320e+06  6.477095e-08   5560.858097   
verified       8179.0  8.697047e+04  6.418341e+05  3.057618e-03    171.576166   

                        50%           75%           max  
sc_type                                                  
non_verified  327481.753543  2.362669e+06  6.957581e+07  
not_sc         36244.911277  2.011576e+05  2.866998e+08  
verified        4538.378425  3.925725e+04  4.518394e+07  
                count          mean           std           min           25%  \
sc_type                                                                         
non_verified    181.0  1.812395e+06  6.077982e+06  6.356919e+00  14211.593387   
not_sc        43333.0  1.643936e+05  1.536418e+06  2.950584e-

In [24]:
with_sc_type.columns

Index(['pool_addr', 'liquidity_provider', 'week', 'overall_roi', 'amt_roi',
       'fee_roi', 'fee', 'amt', 'amt_last', 'amount_input', 'amount_output',
       'position_cnt', 'active_perc', 'sc_usage_avg', 'rec_usage_avg',
       'lp_type', 'mean_tx_fee', 'sum_tx_fee', 'cum_in', 'cum_overall_roi',
       'sc_type'],
      dtype='object')

In [50]:
data_given_pool

,pool_addr,liquidity_provider,week,overall_roi,amt_roi,fee_roi,fee,amt,amt_last,amount_input,...,cum_in,last_week,last_two_week,last_three_week,last_four_week,continue_flag_con1,continue_flag_con2,continue_flag_con3,continue_flag_con4,cum_overall_roi
174156,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x00000000000b4bdf11195d2a010c32a91b8af06a,2022-07-24,1.046423,1.038826,0.007598,0.184905,25.281673,24.336780,0.000000,...,21.261458,2022-07-17,2022-07-10,2022-07-03,2022-06-26,True,True,True,True,1.046423
174157,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x00000000000b4bdf11195d2a010c32a91b8af06a,2022-07-31,1.004022,0.999783,0.004239,0.107167,25.276196,25.281673,0.000000,...,21.261458,2022-07-24,2022-07-17,2022-07-10,2022-07-03,True,True,True,True,1.050633
174158,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x00000000000b4bdf11195d2a010c32a91b8af06a,2022-08-07,1.063434,1.059542,0.003891,0.098362,26.781202,25.276196,0.000000,...,21.261458,2022-07-31,2022-07-24,2022-07-17,2022-07-10,True,True,True,True,1.117278
174159,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x00000000000b4bdf11195d2a010c32a91b8af06a,2022-08-14,0.919410,0.915265,0.004145,0.111010,24.511887,26.781202,0.000000,...,21.261458,2022-08-07,2022-07-31,2022-07-24,2022-07-17,True,True,True,True,1.027237
174160,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x00000000000b4bdf11195d2a010c32a91b8af06a,2022-08-21,0.980812,0.976096,0.004716,0.115604,23.925960,24.511887,0.000000,...,21.261458,2022-08-14,2022-08-07,2022-07-31,2022-07-24,True,True,True,True,1.007526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898985,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xfffa09b517354968d378573c9a31d1586fd8792d,2022-11-27,1.008945,1.007554,0.001390,6.468989,4688.543698,4653.389820,3559.118161,...,126994.264712,2022-11-20,2022-11-13,2022-11-06,2022-10-30,True,True,True,True,1.037017
898986,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xfffa09b517354968d378573c9a31d1586fd8792d,2022-12-04,0.997288,0.996303,0.000984,4.719385,4776.544269,4794.266326,3620.146620,...,130614.411332,2022-11-27,2022-11-20,2022-11-13,2022-11-06,True,True,True,True,1.034204
898987,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xfffa09b517354968d378573c9a31d1586fd8792d,2022-12-11,0.987953,0.987276,0.000677,4.121718,6010.390759,6087.855674,4907.866604,...,135522.277936,2022-12-04,2022-11-27,2022-11-20,2022-11-13,True,True,True,True,1.021745
898988,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xfffa09b517354968d378573c9a31d1586fd8792d,2022-12-18,1.006439,1.005251,0.001188,2.957339,2501.423567,2488.357632,1257.350996,...,136779.628932,2022-12-11,2022-12-04,2022-11-27,2022-11-20,True,True,True,True,1.028324


In [49]:
#with_sc_type["sc_type"].value_counts()

sc_type
not_sc      287638
verified     51626
Name: count, dtype: int64

In [51]:
data["liquidity_provider"].nunique()

17386

In [27]:
datadata_given_pool.shape

(52002, 28)

In [28]:
data.shape

(94309, 27)

In [55]:
data.groupby(["lp_type"])["overall_roi"].mean()

lp_type
manual_only             1.000104
rec_and_manual_mixed    0.997628
rec_only                0.998020
sc_only                 0.999247
Name: overall_roi, dtype: float64

In [56]:
data.to_stata("/Users/allen/Downloads/test0117.dta")

In [29]:
data_bak.shape

(899087, 19)